In [10]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data.shape, test.shape

((2586, 3), (288, 2))

In [7]:
data = pd.merge(data, test,how='left')

In [8]:
data.shape

(2586, 3)

# TfidVectorizer with SGDC 

In [11]:
# Pipeline

vect = TfidfVectorizer(stop_words='english')
scaler = StandardScaler(with_mean=False, with_std=False)
sgdc = SGDClassifier(alpha=0.0007, 
                     average=False, 
                     class_weight='balanced',
                     early_stopping=False, 
                     epsilon=0.01, 
                     eta0=0.1, 
                     fit_intercept=True,
                     l1_ratio=0.15, 
                     learning_rate='optimal', 
                     loss='hinge', 
                     max_iter=500,
                     n_iter_no_change=8, 
                     n_jobs=None,  
                     penalty='l2', 
                     power_t=0.5,
                     random_state=69, 
                     shuffle=True, 
                     tol=0.0015, 
                     validation_fraction=0.05,
                     verbose=0, 
                     warm_start=True)

pipe = Pipeline([('vect', vect), 
                 ('scaler', scaler),
                 ('clf', sgdc)])
# Fit Pipeline
pipe.fit(data.description, data.category)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=69, shuffle=True, tol=0.0015,
       validation_fraction=0.05, verbose=0, warm_start=True))])

In [12]:
parameters = {
    'clf__tol' : (.0015, 1, 2, 3, 4, .05, .005)

}

In [13]:
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)


In [14]:
grid_search.fit(data.description, data.category)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    4.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=69, shuffle=True, tol=0.0015,
       validation_fraction=0.05, verbose=0, warm_start=True))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__tol': (0.0015, 1, 2, 3, 4, 0.05, 0.005)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [15]:
grid_search.best_params_

{'clf__tol': 0.0015}

In [16]:
grid_search.best_score_

0.9404485692188709

In [17]:
pred = grid_search.predict(test['description'])

In [18]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [19]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,4
3,1024,1
4,1902,1


In [20]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('submission4_W.csv', index=False)